In [1]:
import os
import pyspark.sql.functions as F
from pyspark.sql.functions import col, lit, lag
from pyspark.sql.window import Window
from pyspark.sql.types import LongType 


In [2]:
results_path = "/results/"
start_date='2020-01-01'
end_date='2020-04-16'
source='cuebiq'
country='ID'
admin_id='ADM4_PCODE'
start_hour_day = 8
end_hour_day = 20
# city_id='UC_NM_MN'
# demographics=[
# 'wealth_index',
# 'population_density',
# ]

# q_days=0.5
# min_pings=20

# Compute share of activity in unique neighborhoods

In [4]:
pings_geocoded = spark.read.parquet(os.path.join(results_path,source, 'processed',country,'pings_geocoded')).withColumn("date",  F.date_format('time','yyyy-MM-dd')).withColumn("hour",  F.hour('time'))
#.drop('accuracy','ADM4_PCODE')
pings_geocoded = pings_geocoded.where((pings_geocoded.time >= start_date) & (pings_geocoded.time <= end_date))


In [5]:
n_admin_date_id=pings_geocoded.groupby(['date','cuebiq_id']).agg(F.countDistinct(admin_id).alias("nunique"), F.count(lit(1)).alias("n_pings"))

day_df = pings_geocoded.filter((col("hour") >= 8) & (col("hour") < 20))
n_admin_daytime_id = day_df.groupby(['date','cuebiq_id']).agg(F.countDistinct(admin_id).alias("nunique"), F.count(lit(1)).alias("n_pings"))

night_df  = pings_geocoded.filter(((col("hour") >= 20) & (col("hour") <= 23)) |((col("hour") >= 0) & (col("hour") < 8)))
n_admin_nighttime_id = night_df.groupby(['date','cuebiq_id']).agg(F.countDistinct(admin_id).alias("nunique"), F.count(lit(1)).alias("n_pings"))


In [6]:
n_admin_date_id = n_admin_date_id.withColumn('pct_unique', col('nunique') / col('n_pings'))
n_admin_nighttime_id = n_admin_nighttime_id.withColumn('pct_unique', col('nunique') / col('n_pings'))
n_admin_daytime_id = n_admin_daytime_id.withColumn('pct_unique', col('nunique') / col('n_pings'))

In [7]:

results_path = '/dbfs/results/'

#%%


n_admin_date_id.toPandas().to_csv(os.path.join(results_path,source,'processed',country,'n_admin_date_id.csv'), index=False)
n_admin_daytime_id.toPandas().to_csv(os.path.join(results_path,source,'processed',country,'n_admin_daytime_id.csv'), index = False)
n_admin_nighttime_id.toPandas().to_csv(os.path.join(results_path,source,'processed',country,'n_admin_nighttime_id.csv'), index = False)


# Figures